<a href="https://colab.research.google.com/github/itsmerajesh4990/AIpracticeandtraining/blob/main/ENEC_rag_vector_stores_pineconeindexdemo_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone Vector Store

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index llama-index-vector-stores-pinecone

In [ ]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Creating a Pinecone Index

In [ ]:
from pinecone import Pinecone, ServerlessSpec

In [ ]:
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Google Colab secrets
openai.api_key = userdata.get('openai')

if openai.api_key:
    os.environ["OPENAI_API_KEY"] = openai.api_key

In [ ]:
#PUT Tyour api key

from google.colab import userdata


api_key = userdata.get('PINECONE_API_KEY')

if api_key:
    os.environ["PINECONE_API_KEY"] = api_key


pc = Pinecone(api_key=api_key)

In [ ]:
api_key

In [ ]:
index_name = "quickstart"

# Delete index if exists
if index_name in [idx["name"] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# dimensions are for text-embedding-ada-002
pc.create_index(
    name=index_name,
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [ ]:
pinecone_index = pc.Index("quickstart")

#### Load documents, build the PineconeVectorStore and VectorStoreIndex

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display

Download Data

In [ ]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
documents

The previous error occurred because the variable `pinecone_index` was used before it was defined. To fix this, I have combined the code that defines `pinecone_index` and the code that uses it into a single cell.

In [ ]:
# Initialize Pinecone index
pinecone_index = pc.Index("quickstart")

# initialize without metadata filter
from llama_index.core import StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

#### Query Index

May take a minute or so for the index to be ready!

In [ ]:
# set Logging to DEBUG for more detailed outputs
import time
from IPython.display import Markdown, display

query_engine = index.as_query_engine()
start_time = time.time()
response = query_engine.query("what is this document about")
display(Markdown(f"<b>{response}</b>"))

# End timer and print duration
end_time = time.time()
print(f"\nExecution Time: {end_time - start_time:.2f} seconds")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

In [ ]:
pip install gradio

In [ ]:

# --- Imports ---
import os
import logging
import sys
import gradio as gr
from IPython.display import Markdown, display

from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# New import for OpenAI LLM
from llama_index.llms.openai import OpenAI

# --- Initialize Pinecone ---

index_name = "quickstart"
dimension = 1536

# Delete index if exists (optional: mirrors original behavior)
if index_name in [idx["name"] for idx in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load Data ---
# Create folders & download a sample doc (kept same logic, fixed subfolder creation)

documents = SimpleDirectoryReader("./data").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Initialize OpenAI LLM
llm = OpenAI(model="gpt-5.2-2025-12-11", api_key=os.environ["OPENAI_API_KEY"])

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, llm=llm
)

# --- System Prompt (polite + answer-from-document constraint) ---
SYSTEM_PROMPT = """You are Aisha, a polite and professional Insurance assistant.
Answer ONLY using the information found in the indexed insurance document(s).
If the answer is not in the document(s), say: "I couldn’t find that in the document."
Keep responses concise, helpful, and courteous.
"""

# --- Query Engine ---
query_engine = index.as_query_engine()

def query_doc(user_question: str):
    if not user_question or not user_question.strip():
        return "Please enter a question."
    full_query = f"""{SYSTEM_PROMPT}

User question:
{user_question.strip()}
"""
    try:
        response = query_engine.query(full_query)
        text = str(response).strip()
        # Gentle post-processing to keep it brief/polite
        return text if text else "I couldn’t find that in the document."
    except Exception as e:
        return f"Error: {str(e)}"

# --- Gradio UI (Professional look with logo, centered title) ---
# Use the raw GitHub URL for proper image rendering.
LOGO_URL = "https://raw.githubusercontent.com/itsmerajesh4990/AIpracticeandtraining/main/ENEC%20logo%20new.jpeg"
CUSTOM_CSS = """
.gradio-container { font-family: Inter, ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, 'Helvetica Neue', Arial; }
.header-wrap {
    display: grid;
    grid-template-columns: 120px 1fr 120px;
    align-items: center;
    gap: 12px;
    padding: 12px 0 8px;
    border-bottom: 1px solid #eaeaea;
}
.header-logo { display:flex; align-items:center; justify-content:flex-start; }
.header-logo img { height: 48px; object-fit: contain; }
.header-title { text-align:center; }
.header-title h1 {
    margin: 0; font-weight: 700; font-size: 1.5rem; line-height: 1.2;
}
.header-spacer { height: 1px; }
.section { padding-top: 8px; }
.footer-note { text-align:center; font-size: 12px; color:#667085; padding: 8px 0 0; }
label.svelte-1ipelgc, .label-wrap label { font-weight: 600; }
"""

with gr.Blocks(css=CUSTOM_CSS, title="SOP Document QA (LlamaIndex + Pinecone)") as demo:
    # Header with logo (left) and centered title
    with gr.Row(elem_classes="header-wrap"):
        with gr.Column(scale=0, elem_classes="header-logo"):
            gr.HTML(f'<img src="{LOGO_URL}" alt="ENEC Logo" />')
        with gr.Column(scale=1, elem_classes="header-title"):
            gr.HTML("<h1>Insurance QA</h1>")
        with gr.Column(scale=0):
            gr.HTML("")  # right-side spacer

    gr.Markdown(
        "Ask questions based on the SOP Document "
        "**Answers come only from the document**. If not found, I’ll say so."
    )

    with gr.Group(elem_classes="section"):
        inp = gr.Textbox(
            label="Your question",
            placeholder="e.g., Ask SOP Question?",
            lines=2,
        )
        btn = gr.Button("Submit", variant="primary")
        out = gr.Textbox(label="Answer", lines=8)

    btn.click(fn=query_doc, inputs=inp, outputs=out)
    inp.submit(fn=query_doc, inputs=inp, outputs=out)

    gr.Markdown('<div class="footer-note">LlamaIndex + Pinecone • Demo</div>')

demo.launch()

Replicate the Deplpyment hugging face
https://huggingface.co/spaces/decodingdatascience/ddsinsurance1
openai key


In [ ]:
documents

In [ ]:
# --- Imports ---
import os
import logging
import sys
import gradio as gr
from IPython.display import Markdown, display

from pinecone import Pinecone, ServerlessSpec
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

# OpenAI LLM
from llama_index.llms.openai import OpenAI

# --- Logging ---
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# --- Initialize Pinecone ---
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "quickstart"
dimension = 1536

# Delete index if exists
existing_indexes = [idx["name"] for idx in pc.list_indexes()]
if index_name in existing_indexes:
    pc.delete_index(index_name)

# Create Pinecone index
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

pinecone_index = pc.Index(index_name)

# --- Load SOP Document ---
# Place your SOP file inside ./data
documents = SimpleDirectoryReader("./data").load_data()

# --- Create Index ---
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
)

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    llm=llm,
)

# --- System Prompt for SOP ---
SYSTEM_PROMPT = """
You are a technical assistant specializing in industrial equipment.
Answer ONLY using the information found in the indexed SOP document:
'SOP_ Troubleshooting Overheated Boiler Feed Pump (Pump P-201)'.

If the answer is not in the document, say:
"I couldn’t find that in the document."

Keep responses concise, accurate, and professional.
"""

# --- Query Engine ---
query_engine = index.as_query_engine(
    system_prompt=SYSTEM_PROMPT
)

def query_doc(user_question: str):
    if not user_question or not user_question.strip():
        return "Please enter a question."

    try:
        response = query_engine.query(user_question.strip())
        text = str(response).strip()

        if not text:
            return "I couldn’t find that in the document."

        return text
    except Exception as e:
        return f"Error: {str(e)}"

# --- Gradio UI ---
LOGO_URL = "https://raw.githubusercontent.com/itsmerajesh4990/AIpracticeandtraining/main/ENEC%20logo%20new.jpeg"

CUSTOM_CSS = """
.gradio-container { font-family: Inter, ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, 'Helvetica Neue', Arial; }
.header-wrap {
    display: grid;
    grid-template-columns: 120px 1fr 120px;
    align-items: center;
    gap: 12px;
    padding: 12px 0 8px;
    border-bottom: 1px solid #eaeaea;
}
.header-logo { display:flex; align-items:center; justify-content:flex-start; }
.header-logo img { height: 48px; object-fit: contain; }
.header-title { text-align:center; }
.header-title h1 {
    margin: 0; font-weight: 700; font-size: 1.5rem; line-height: 1.2;
}
.section { padding-top: 8px; }
.footer-note { text-align:center; font-size: 12px; color:#667085; padding: 8px 0 0; }
"""

with gr.Blocks(css=CUSTOM_CSS, title="SOP QA (LlamaIndex + Pinecone)") as demo:
    with gr.Row(elem_classes="header-wrap"):
        with gr.Column(scale=0, elem_classes="header-logo"):
            gr.HTML(f'<img src="{LOGO_URL}" alt="ENEC Logo" />')
        with gr.Column(scale=1, elem_classes="header-title"):
            gr.HTML("<h1>SOP Troubleshooting QA</h1>")
        with gr.Column(scale=0):
            gr.HTML("")

    gr.Markdown(
        "Ask questions based on the SOP document for Pump P‑201. "
        "**Answers come only from the document**."
    )

    with gr.Group(elem_classes="section"):
        inp = gr.Textbox(
            label="Your question",
            placeholder="e.g., What is the first step when the pump overheats?",
            lines=2,
        )
        btn = gr.Button("Submit", variant="primary")
        out = gr.Textbox(label="Answer", lines=8)

    btn.click(fn=query_doc, inputs=inp, outputs=out)
    inp.submit(fn=query_doc, inputs=inp, outputs=out)

    gr.Markdown('<div class="footer-note">LlamaIndex + Pinecone • Demo</div>')

demo.launch()